In [13]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd


df = pd.read_csv('alugueis.csv')

In [15]:
def amostragem_estratificada(df, amostras, coluna):
    column_values = df[coluna].value_counts().items()
    df_length = len(df)

    percentages = {}
    for city, number_of_ocurrences in column_values:
        percentage = number_of_ocurrences / df_length
        percentages[city] = percentage

    samples_df = pd.DataFrame()
    for city, percentage in percentages.items():
        n = int(percentage * amostras)
        df_city = df[df[coluna] == city]
        samples_df = pd.concat([samples_df, df_city.sample(n)])
        
    return samples_df

new_df = amostragem_estratificada(df, 100, 'city')

print(df.size)
print(new_df['city'].value_counts(normalize=True))
print(df['city'].value_counts(normalize=True))

148554
city
São Paulo         0.555556
Rio de Janeiro    0.141414
Belo Horizonte    0.111111
Porto Alegre      0.111111
Campinas          0.080808
Name: proportion, dtype: float64
city
São Paulo         0.550655
Rio de Janeiro    0.141174
Belo Horizonte    0.116294
Porto Alegre      0.111582
Campinas          0.080294
Name: proportion, dtype: float64


In [16]:
amostragem_estratificada(df, 200, 'city')

,id,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
6182,6234,São Paulo,50,1,1,0,-,acept,not furnished,0,700,67,11,778
9978,10050,São Paulo,600,4,6,5,13,acept,not furnished,7200,6000,1167,77,14440
170,171,São Paulo,240,4,4,3,9,acept,not furnished,2500,2144,834,28,5506
6916,6973,São Paulo,160,3,3,1,-,acept,not furnished,0,8000,0,121,8121
8831,8897,São Paulo,180,3,4,3,15,acept,not furnished,2200,8900,782,113,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10464,10544,Campinas,202,3,2,4,-,acept,not furnished,0,3500,608,53,4161
10530,10611,Campinas,411,4,5,4,-,acept,not furnished,0,5500,667,83,6250
8966,9032,Campinas,68,1,1,1,10,not acept,furnished,600,1000,48,13,1661
5620,5665,Campinas,45,1,1,1,2,acept,not furnished,367,1100,34,14,1515


In [17]:
import sys
areas = pd.cut(df["area"], bins=[0, 50, 100, sys.maxsize], labels=["PEQUENO", "MÉDIO", "GRANDE"])

areas.value_counts()


area
GRANDE     4701
MÉDIO      3655
PEQUENO    2255
Name: count, dtype: int64

In [18]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [24]:
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
df["total_discretized"] = discretizer.fit_transform(df[["total"]])

encoder = OneHotEncoder(sparse_output=False)
onehot = encoder.fit_transform(df[['total_discretized']])

onehot_df = pd.DataFrame(onehot, columns=[f'total_discretized_{i}' for i in range(5)])

result = pd.concat([df, onehot_df], axis=1)

print("\nInformações sobre as categorias:")
for i, (lower, upper) in enumerate(zip(discretizer.bin_edges_[0][:-1], discretizer.bin_edges_[0][1:])):
    print(f"Categoria {i}: {lower:.2f} até {upper:.2f}")


Informações sobre as categorias:
Categoria 0: 499.00 até 1823.00
Categoria 1: 1823.00 até 2861.00
Categoria 2: 2861.00 até 4442.00
Categoria 3: 4442.00 até 7969.00
Categoria 4: 7969.00 até 22630.00
